# Train model using Mouse Brain dataset

In this notebook, we're going to train our model using the Mouse Brain dataset (GSE60361). 

This assumes that you've made the graph using the ```Infer GRN.ipynb``` code.

In [1]:
import os

import numpy as np
import pandas as pd
import torch
import torch_geometric
from torch_geometric.data import Data, Dataset
from tqdm import tqdm
from datasets.datasetAtacSeq import AtacSeqDataset
from scipy.special import softmax
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from tqdm import tqdm
from sklearn.metrics import (auc, precision_recall_curve, roc_auc_score,
                             roc_curve)
from statistics import mean

Torch version: 1.8.0+cu111
Cuda available: True
Torch geometric version: 2.0.3


Load up the dataset. Read ```datasetMouseBrain.py``` on how the dataset was built. 

In [2]:
dataset = AtacSeqDataset("/gpfs/data/rsingh47/hzaki1/atacseqdata")

100%|██████████| 103480/103480 [00:39<00:00, 2597.98it/s]


In [3]:
print()
print(f'Dataset: {dataset}:')
print('====================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')

data = dataset[0]  # Get the first graph object.

print()
print(data)
print('=============================================================')

# Gather some statistics about the first graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Contains isolated nodes: {data.contains_isolated_nodes()}')
print(f'Contains self-loops: {data.contains_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')


Dataset: AtacSeqDataset(1047):
Number of graphs: 1047
Number of features: 1

Data(x=[18666], edge_index=[2, 103480], y=[1])
Number of nodes: 18666
Number of edges: 103480
Average node degree: 5.54
Contains isolated nodes: True
Contains self-loops: False
Is undirected: False


/gpfs/home/hzaki1/celltypefromgrn/env-gpu/lib/python3.7/site-packages/torch_geometric/deprecation.py:13: UserWarning: 'contains_isolated_nodes' is deprecated, use 'has_isolated_nodes' instead
  warnings.warn(out)
/gpfs/home/hzaki1/celltypefromgrn/env-gpu/lib/python3.7/site-packages/torch_geometric/deprecation.py:13: UserWarning: 'contains_self_loops' is deprecated, use 'has_self_loops' instead
  warnings.warn(out)


In [8]:
rangeNp = np.arange(0, 1047)
np.random.shuffle(rangeNp)

In [10]:
print(rangeNp)
np.savetxt('shuffle_indices/atacseqShuffleIndex.txt', rangeNp)

[936 657 512 ... 356 237  18]


In [11]:
torch.manual_seed(12345)
dataset = dataset.shuffle()


shuffle_index = np.loadtxt('shuffle_indices/atacseqShuffleIndex.txt')
shuffle_index = shuffle_index.astype(np.int32)
train_size, val_size = int(len(shuffle_index)* 0.8), int(len(shuffle_index)* 0.9)
train_dataset = [dataset[i] for i in shuffle_index[0:train_size]]
val_dataset = [dataset[i] for i in shuffle_index[train_size: val_size]]
test_dataset =  [dataset[i] for i in shuffle_index[val_size:]]

# train_dataset = torch.load('trainDataset.pt')
# test_dataset = torch.load('testDataset.pt')

# train_dataset = dataset[:837]
# test_dataset = dataset[837:]

# train_dataset = dataset[:40]
# test_dataset = dataset[40:60]

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of validation graphs: {len(val_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')

Number of training graphs: 837
Number of validation graphs: 105
Number of test graphs: 105


In [12]:
from torch_geometric.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=30, shuffle=True)
train_loader_testing = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

/gpfs/home/hzaki1/celltypefromgrn/env-gpu/lib/python3.7/site-packages/torch_geometric/deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [13]:
from gcnmodel import GCN
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = GCN(hidden_channels=128, data=dataset, output_size=4).to(device)
print(model)

GCN(
  (conv1): SAGEConv(1, 128)
  (conv2): SAGEConv(128, 128)
  (conv3): SAGEConv(128, 128)
  (lin): Linear(in_features=128, out_features=4, bias=True)
)


In [14]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
criterion = torch.nn.CrossEntropyLoss()
    
def test(loader, size):
    model.eval()
    output = np.zeros((len(loader), size))
    actual = np.zeros((len(loader), size))
    accuracy = 0
    for ind, data in enumerate(loader):  # Iterate in batches over the training/test dataset.
        data.x = torch.reshape(data.x, (data.x.shape[0], 1))
        data.x = data.x.type(torch.FloatTensor)
        data = data.to(device)
        out = model(data.x, data.edge_index, data.batch)
        output[ind] = softmax(out.cpu().detach().numpy())
        actual[ind][data.y] = 1
        accuracy += int((out.argmax(dim=1) == data.y).sum())
    all_labels = list(dataset.cellToIndex.keys())
    actual = np.array(actual)
    precision = dict()
    recall = dict()
    averageAUROC = []
    averageAUPR = []
    for (idx, c_label) in enumerate(all_labels):
        
        fpr, tpr, thresholds = roc_curve(actual[:,idx].astype(int), output[:,idx])
        precision[idx], recall[idx], _ = precision_recall_curve(actual[:, idx],
                                                        output[:, idx])
        averageAUROC.append(auc(fpr, tpr))
        averageAUPR.append(round(auc(recall[idx], precision[idx]),4))

    return accuracy/len(loader.dataset), mean(averageAUROC), mean(averageAUPR)


def train():
    model.train()
    avgLoss = 0
    for data in tqdm(train_loader, total=30):  # Iterate in batches over the training dataset.
        data.x = torch.reshape(data.x, (data.x.shape[0], 1))
        data.x = data.x.type(torch.FloatTensor)
        data = data.to(device)
        out = model(data.x, data.edge_index, data.batch)# Perform a single forward pass.
        loss = criterion(out, data.y)  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.
        avgLoss += loss
    return avgLoss / 30


for epoch in range(1, 150):
        loss = train()
        train_acc, trainAUC, trainAUPR = test(train_loader_testing, 4)
        test_acc,testAUC, testAUPR = test(val_loader, 4)
        print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Train AUC: {trainAUC:.4f}, Train AUPR: {trainAUPR:.4f}, Test Acc: {test_acc:.4f}, Test Auc: {testAUC:.4f}, Test AUPR: {testAUPR:.4f},  Loss: {loss:.4f}')

 93%|█████████▎| 28/30 [00:06<00:00,  4.23it/s]


Epoch: 001, Train Acc: 0.3572, Train AUC: 0.5877, Train AUPR: 0.3096, Test Acc: 0.3714, Test Auc: 0.6010, Test AUPR: 0.3267,  Loss: 1.2530


 93%|█████████▎| 28/30 [00:04<00:00,  6.47it/s]


Epoch: 002, Train Acc: 0.3740, Train AUC: 0.6363, Train AUPR: 0.3471, Test Acc: 0.3905, Test Auc: 0.6580, Test AUPR: 0.3781,  Loss: 1.2367


 93%|█████████▎| 28/30 [00:07<00:00,  3.94it/s]


...

Epoch: 147, Train Acc: 0.6738, Train AUC: 0.8723, Train AUPR: 0.7077, Test Acc: 0.6190, Test Auc: 0.8329, Test AUPR: 0.5831,  Loss: 0.8168


 93%|█████████▎| 28/30 [00:07<00:00,  3.93it/s]


Epoch: 148, Train Acc: 0.6643, Train AUC: 0.8668, Train AUPR: 0.6877, Test Acc: 0.6667, Test Auc: 0.8018, Test AUPR: 0.5625,  Loss: 0.8306


 93%|█████████▎| 28/30 [00:09<00:00,  3.09it/s]


Epoch: 149, Train Acc: 0.6774, Train AUC: 0.8700, Train AUPR: 0.7009, Test Acc: 0.6286, Test Auc: 0.8095, Test AUPR: 0.5701,  Loss: 0.8491


In [15]:
torch.save(model.state_dict(), 'model_weightsjun3_atacseq_data.pth')

In [9]:
!nvidia-smi

Fri Mar 11 23:20:13 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:24:00.0 Off |                  N/A |
| 62%   62C    P2   249W / 350W |  13415MiB / 24268MiB |     44%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------